In [1]:
from json import JSONDecodeError

from google import genai
import json

from Gemini import IndicatorExtraction

client = genai.Client()

In [2]:
from Gemini import IndicatorExtraction
#Need: company_name, year, indicator_id, not_disclosed, value, unit, page_number, source_title (CompanyName_Year_Topic_Counter), text section, input token count, output token count, thoughts
def parse_response(response_json):
    responseData_dic = {}

    doc_properties = response_json['key'].split('-')
    responseData_dic['company_name'] = doc_properties[0]
    responseData_dic['year'] = doc_properties[1]
    responseData_dic['indicator_id'] = doc_properties[4]
    responseData_dic['source_title'] = doc_properties[0] + "_" + doc_properties[1] + "_" + doc_properties[2] + "_" + doc_properties[3]

    print(responseData_dic['source_title'])

    response_text = ""
    response_parts = response_json['response']['candidates'][0]['content']['parts']
    for response_part in response_parts:
        if 'thought' in response_part:
            responseData_dic['thoughts'] = response_part['text']
        else:
            response_text = response_part['text']

    response: IndicatorExtraction = json.loads(response_text)
    if "isDisclosed" not in response:
        #print(f"isDisclosed not specified! Setting it to 1")
        if "not disclosed" in response['value'].lower() or "n/a" in response['value'].lower():
            response["isDisclosed"] = 0
        else:
            response["isDisclosed"] = 1


    responseData_dic['isDisclosed'] = response['isDisclosed']
    responseData_dic['value'] = response['value']
    responseData_dic['unit'] = response['unit']
    responseData_dic['page_number'] = response['page_number']
    responseData_dic['section'] = response['section']

    responseData_dic['inputTokenCount'] = response_json['response']["usageMetadata"]["promptTokenCount"]
    responseData_dic['outputTokenCount'] = response_json['response']["usageMetadata"]["candidatesTokenCount"] + response_json['response']["usageMetadata"]["thoughtsTokenCount"]

    return responseData_dic

In [7]:
from MySQL_client import insertIntoBatchMetricExtraction
import json

# The output is in another file.
result_file_name = "files/batch-sy6qfpgtt4a9myxdal4pzscjwd4srs155156"
print(f"Results are in file: {result_file_name}")

print("\nDownloading and parsing result file content...")
file_content_bytes = client.files.download(file=result_file_name)
file_content = file_content_bytes.decode('utf-8')

parsed_response = ""

# The result file is also a JSONL file. Parse and print each line.
for line in file_content.splitlines():
  if line:
    try:
        responseData_dic = parse_response(json.loads(line))
        insertIntoBatchMetricExtraction(responseData_dic)
    except json.JSONDecodeError as e:
        print(e)

Results are in file: files/batch-sy6qfpgtt4a9myxdal4pzscjwd4srs155156

Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
Ford_2024_ANNUAL_REPORT_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
GeneralMotors_2020_ESG_1
Genera

KeyError: 'content'